# 0. Imports

In [ ]:
# Imports
from IPython.display import display, HTML
import os
import pandas as pd, datetime as dt, numpy as np, matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
import sys

# Display options
thisnotebooksys = sys.stdout
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

In [ ]:
import mimicLOB as mlob

## 1. LOB creation

In [ ]:
# b_tape = True means the LOB 
LOB = mlob.OrderBook(tick_size  = 0.5, 
                     b_tape     = True, # tape transactions
                     b_tape_LOB = True, # tape lob state at each tick
                     verbose    = True)

## 2. Data

- DTIME : le timestamp de l'ordre
- ORDER_ID : l'identifiant de l'ordre
- PRICE 
- QTY
- ORDER_SIDE
- ORDER_SIDE
- ORDER_TYPE :  <br>1 pour Market Order; <br>2 pour Limit Order; <br>q pour Quote <br> W pour Market On Open;
- ACTION_TYPE : <br> I = limit order insertion (passive); <br> C = limit order cacnellations; <br> R = replace order that lose priority; <br> r = replace order that keeps priority; <br> S = replace order that makes the order aggressive (give rise to trade); <br> T = aggressive order (give rise to trade)
- MATCH_STRATEGY : True/False
- IS_OPEN_TRADE :  True/False

In [ ]:
df = pd.read_pickle(r'..\data\day20160428.pkl')
df

## 3. Agents Creation

In [ ]:
auction_config = {'orderbook'        : LOB,
                  'id'               : 'FDR',
                  'b_record'         : False,
                  'historicalOrders' : df[df.DTIME.dt.hour<7]}

continuousTrading_config = {'orderbook'        : LOB,
                            'id'               : 'FDR',
                            'b_record'         : False,
                            'historicalOrders' : df[df.DTIME.dt.hour>=7]}

AuctionReplayer = mlob.replayerAgent(**auction_config)
ContReplayer    = mlob.replayerAgent(**continuousTrading_config)

## 4. Replay orders

### 4.1. Auction phase
#### The auction price shall be determined on the basis of the situation of the Central Order Book at the closing of the call phase and shall be the price which produces the highest executable order volume.

In [ ]:
%%time

# log
f = open('log_auction.txt','w');
sys.stdout = f

# Close auction
LOB.b_auction = True #Open auction
AuctionReplayer.replayOrders()

# log
sys.stdout = thisnotebooksys

### 4.2. Auction is over

Closing the auction will result in transactions and a new LOB with unmatched orders will be set.
The price is chosen as the one that maximizes volume of transactions.

Trades are executed at the auction price, and according to a time priority. The remaining orders at the auction price are the newest orders.

In [ ]:
%%time

# log
f = open('log_auctionClose.txt','w');
sys.stdout = f

LOB.b_auction = False

# log
sys.stdout = thisnotebooksys

### 4.3. Lob State
LOB state before opening the continuous trading

In [ ]:
LOBstate = AuctionReplayer.getLOBState()
LOBstate = LOBstate.set_index('Price').sort_index()
LOBstate.plot.bar(figsize=(20, 7))
plt.show()

### 4.4. Continuous Trading

In [ ]:
%%time

# log
f = open('log_continuousTrading.txt','w');
sys.stdout = f

# Close auction
ContReplayer.replayOrders()

# log
sys.stdout = thisnotebooksys

## 5. Price Tape

In [ ]:
histoPrices = ContReplayer.getPriceTape().astype(float)
histoPrices.plot(figsize=(20,7))

# OHLC
display(f'open  : {histoPrices.iloc[0,0]}')
display(f'high  : {histoPrices.max()[0]}')
display(f'low   : {histoPrices.min()[0]}')
display(f'close : {histoPrices.iloc[-1, 0]}')

plt.show()

## Get Transaction Tape

In [ ]:
TransactionTape = ContReplayer.getTransactionTape()

In [ ]:
TransactionTape

## Get LOB Tape 

The LOB tape is the state of the LOB before each order arrival

In [ ]:
LOBtape = AuctionReplayer.getLOBTape()

In [ ]:
LOBtape